In [1]:
import pandas as pd
df = pd.read_csv('train.csv')
TARGETS = df.columns[-6:]
print('Train shape:', df.shape )
print('Targets', list(TARGETS))
df.head()


Train shape: (106800, 15)
Targets ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']


,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,1628180742,0,0.0,353733,0,0.0,127492639,42516,Seizure,3,0,0,0,0,0
1,1628180742,1,6.0,353733,1,6.0,3887563113,42516,Seizure,3,0,0,0,0,0
2,1628180742,2,8.0,353733,2,8.0,1142670488,42516,Seizure,3,0,0,0,0,0
3,1628180742,3,18.0,353733,3,18.0,2718991173,42516,Seizure,3,0,0,0,0,0
4,1628180742,4,24.0,353733,4,24.0,3080632009,42516,Seizure,3,0,0,0,0,0


In [2]:
train = df.groupby('eeg_id')[['spectrogram_id','spectrogram_label_offset_seconds']].agg(
    {'spectrogram_id':'first','spectrogram_label_offset_seconds':'min'})
train.columns = ['spectrogram_id','min']

tmp = df.groupby('eeg_id')[['spectrogram_id','spectrogram_label_offset_seconds']].agg(
    {'spectrogram_label_offset_seconds':'max'})
train['max'] = tmp

tmp = df.groupby('eeg_id')[['patient_id']].agg('first')
train['patient_id'] = tmp

tmp = df.groupby('eeg_id')[TARGETS].agg('sum')
for t in TARGETS:
    train[t] = tmp[t].values
    
y_data = train[TARGETS].values
y_data = y_data / y_data.sum(axis=1,keepdims=True)
train[TARGETS] = y_data

tmp = df.groupby('eeg_id')[['expert_consensus']].agg('first')
train['target'] = tmp

train = train.reset_index()
print('Train non-overlapp eeg_id shape:', train.shape )

train.head(20)

Train non-overlapp eeg_id shape: (17089, 12)


,eeg_id,spectrogram_id,min,max,patient_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote,target
0,568657,789577333,0.0,16.0,20654,0.000000,0.000000,0.250000,0.000000,0.166667,0.583333,Other
1,582999,1552638400,0.0,38.0,20230,0.000000,0.857143,0.000000,0.071429,0.000000,0.071429,LPD
2,642382,14960202,1008.0,1032.0,5955,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,Other
3,751790,618728447,908.0,908.0,38549,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,GPD
4,778705,52296320,0.0,0.0,40955,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,Other
5,1629671,2036345030,0.0,160.0,37481,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Seizure
6,1895581,128369999,1138.0,1138.0,47999,0.076923,0.000000,0.000000,0.000000,0.076923,0.846154,Other
7,2061593,320962633,1450.0,1450.0,23828,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,Other
8,2078097,2074135650,3342.0,3342.0,61174,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,Other
9,2366870,1232582129,0.0,30.0,23633,0.000000,0.333333,0.000000,0.000000,0.000000,0.666667,Other


In [3]:

import numpy as np

ycol = ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']
cd = {'Seizure': 'seizure_vote', 'GPD': 'gpd_vote', 'LRDA': 'lrda_vote', 'Other': 'other_vote', 'GRDA': 'grda_vote', 'LPD': 'lpd_vote'}


eeg_id_col = train.iloc[:, 0]  
prob_cols = train.iloc[:, -7:-1]  
label_col = train.iloc[:, -1] 


prob_cols = prob_cols.astype("float32")


prob_cols_normalized = prob_cols.div(prob_cols.sum(axis=1), axis=0)


normalized_data = pd.concat([eeg_id_col, prob_cols_normalized, label_col], axis=1)

normalized_data.head(20)

,eeg_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote,target
0,568657,0.000000,0.000000,0.250000,0.000000,0.166667,0.583333,Other
1,582999,0.000000,0.857143,0.000000,0.071429,0.000000,0.071429,LPD
2,642382,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,Other
3,751790,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,GPD
4,778705,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,Other
5,1629671,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Seizure
6,1895581,0.076923,0.000000,0.000000,0.000000,0.076923,0.846154,Other
7,2061593,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,Other
8,2078097,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,Other
9,2366870,0.000000,0.333333,0.000000,0.000000,0.000000,0.666667,Other


In [4]:
normalized_data.to_csv("normalized_data.csv", index=False)


In [5]:
EEG_PATH = 'train_eegs/'
train_path = 'normalized_data.csv'


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import glob
from scipy.signal import butter, sosfilt
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
import time

import pandas as pd

from EGGDataset import EEGDataset
dataset = EEGDataset(train_path, EEG_PATH)


dataloader = DataLoader(dataset, batch_size=64, shuffle=True, num_workers=2, prefetch_factor=2, persistent_workers=True)

In [7]:
X, y = dataset[0]
print(f"Sample {0 + 1}: X shape {X.shape}, y shape {y.shape}")

Sample 1: X shape torch.Size([8, 10000]), y shape torch.Size([6])


In [8]:
import torch.nn.functional as F

class CNNLSTM(nn.Module):

    def __init__(self, in_channels=8, num_classes=6):
        super(CNNLSTM, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu1 = nn.ReLU(inplace=True)
        self.dropout1 = nn.Dropout(p=0.5)
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv1d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm1d(64)
        self.relu2 = nn.ReLU(inplace=True)
        self.dropout2 = nn.Dropout(p=0.75)
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)

        self.lstm1 = nn.LSTM(input_size=64, hidden_size=128, num_layers=1, batch_first=True, bidirectional=True)
        self.lstm2 = nn.LSTM(input_size=256, hidden_size=128, num_layers=1, batch_first=True, bidirectional=True)

        self.attention = nn.Sequential(
            nn.Linear(128 * 2, 64),
            nn.Tanh(),
            nn.Linear(64, 1)
        )

        self.fc = nn.Linear(128 * 2, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.dropout1(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.dropout2(x)
        x = self.pool2(x)

        # reshape for LSTM
        batch_size, channels, seq_length = x.size()
        x = x.permute(0, 2, 1)

        # First LSTM layer
        x, _ = self.lstm1(x)

        # Second LSTM layer
        x, _ = self.lstm2(x)
        
        # Attention layer
        att_weights = F.softmax(self.attention(x), dim=1)
        x = torch.sum(att_weights * x, dim=1)

        # Fully connected layer
        x = self.fc(x)

        return x

In [9]:
input_channels = 8
num_classes = 6  


model = CNNLSTM(in_channels=input_channels, num_classes=num_classes)


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

s = 0
num_epochs = 1 
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(dataloader):
        print(f"Dataloader: {i+1}/{len(dataloader)} - {round(time.time()-s,2)}s")
        s = time.time()
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss / len(dataloader)}")

Dataloader: 1/268 - 1715423508.77s
Dataloader: 2/268 - 22.89s
Dataloader: 3/268 - 18.07s
Dataloader: 4/268 - 19.11s
Dataloader: 5/268 - 16.78s
Dataloader: 6/268 - 16.09s
Dataloader: 7/268 - 16.68s
Dataloader: 8/268 - 17.88s
Dataloader: 9/268 - 22.78s
Dataloader: 10/268 - 18.41s
Dataloader: 11/268 - 20.42s
Dataloader: 12/268 - 17.21s
Dataloader: 13/268 - 16.44s
Dataloader: 14/268 - 18.22s
Dataloader: 15/268 - 18.63s
Dataloader: 16/268 - 17.61s
Dataloader: 17/268 - 17.4s
Dataloader: 18/268 - 20.64s
Dataloader: 19/268 - 16.57s
Dataloader: 20/268 - 16.46s
Dataloader: 21/268 - 16.82s
Dataloader: 22/268 - 29.07s
Dataloader: 23/268 - 20.78s
Dataloader: 24/268 - 17.36s
Dataloader: 25/268 - 16.31s
Dataloader: 26/268 - 16.51s
Dataloader: 27/268 - 16.1s
Dataloader: 28/268 - 16.46s
Dataloader: 29/268 - 16.21s
Dataloader: 30/268 - 16.45s
Dataloader: 31/268 - 16.09s
Dataloader: 32/268 - 16.62s
Dataloader: 33/268 - 23.9s
Dataloader: 34/268 - 29.64s
Dataloader: 35/268 - 19.05s
Dataloader: 36/268 - 14.

In [26]:
num_epochs = 1 
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(dataloader):
        print(f"Dataloader: {i+1}/{len(dataloader)} - {round(time.time()-s,2)}s")
        s = time.time()
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss / len(dataloader)}")

Dataloader: 1/268 - 270.52s
Dataloader: 2/268 - 10.83s
Dataloader: 3/268 - 14.19s
Dataloader: 4/268 - 13.72s
Dataloader: 5/268 - 13.72s
Dataloader: 6/268 - 13.81s
Dataloader: 7/268 - 14.0s
Dataloader: 8/268 - 14.85s
Dataloader: 9/268 - 13.77s
Dataloader: 10/268 - 14.09s
Dataloader: 11/268 - 13.93s
Dataloader: 12/268 - 13.29s
Dataloader: 13/268 - 13.49s
Dataloader: 14/268 - 13.27s
Dataloader: 15/268 - 13.52s
Dataloader: 16/268 - 13.37s
Dataloader: 17/268 - 13.98s
Dataloader: 18/268 - 13.39s
Dataloader: 19/268 - 14.0s
Dataloader: 20/268 - 13.71s
Dataloader: 21/268 - 13.36s
Dataloader: 22/268 - 13.46s
Dataloader: 23/268 - 13.45s
Dataloader: 24/268 - 13.57s
Dataloader: 25/268 - 14.12s
Dataloader: 26/268 - 14.46s
Dataloader: 27/268 - 14.39s
Dataloader: 28/268 - 15.14s
Dataloader: 29/268 - 16.11s
Dataloader: 30/268 - 14.25s
Dataloader: 31/268 - 14.39s
Dataloader: 32/268 - 13.84s
Dataloader: 33/268 - 14.93s
Dataloader: 34/268 - 14.53s
Dataloader: 35/268 - 14.21s
Dataloader: 36/268 - 14.32s
Da

In [29]:
torch.save(model.state_dict(), '3_epoch.pth')

In [27]:
test_path = 'test.csv'
TEST_EEG_PATH = 'test_eegs'

class TestEEGDataset(Dataset):
    def __init__(self, csv_file, eeg_path):
        self.csv = pd.read_csv(csv_file)  # Load the CSV file
        self.eeg_path = eeg_path
        self.sos = self.butter_bandpass_filter_init()  # Initialize the Butterworth filter parameters
        self.FEATS = ['Fp1','T3','C3','O1','Fp2','C4','T4','O2']
        
    def __len__(self):
        return len(self.csv)
    
    def butter_bandpass_filter_init(self):
        lowcut = 0.5  # Set the low-frequency cutoff for bandpass filtering
        highcut = 45.0  # Set the high-frequency cutoff for bandpass filtering
        fs = 200.0  # Sampling frequency
        order = 5  # Filter order

        nyq = 0.5 * fs
        low = lowcut / nyq
        high = highcut / nyq
        sos = butter(order, [low, high], analog=False, btype='band', output='sos')  # Create second-order sections for the Butterworth filter
        return sos

    def butter_bandpass_filter(self, data):
        y = sosfilt(self.sos, data)
        return y

    def __getitem__(self, idx):
        eeg_id = self.csv.loc[idx, 'eeg_id']
        eeg_file_path = "/Users/chloenguyen/Downloads/eeg_data/test_eegs/3911565283.parquet"  # Build the EEG data file path
        eeg_data = pd.read_parquet(eeg_file_path)[self.FEATS].values  # Load EEG data from Parquet file

        eeg_data = self.butter_bandpass_filter(eeg_data)  # Apply filtering to the data
        eeg_data = torch.tensor(eeg_data, dtype=torch.float32)  # Convert to PyTorch tensor
         
        # Select 10,000 data points from the middle
        mid_index = eeg_data.shape[0] // 2
        start_index = mid_index - 5000  # Offset 5000 data points to the left from the middle
        end_index = mid_index + 5000  # Offset 5000 data points to the right from the middle
        eeg_data = eeg_data[start_index:end_index]
        # Swap dimensions
        eeg_data = torch.transpose(eeg_data, 0, 1)
        
        return eeg_data


testdataset = TestEEGDataset(test_path, TEST_EEG_PATH)


test_dataloader = DataLoader(testdataset, batch_size=32, shuffle=True)


model.eval()

predictions = []

with torch.no_grad():
    for inputs in test_dataloader:
        outputs = model(inputs)
      
        probabilities = torch.softmax(outputs, dim=1)
        predictions.append(probabilities.cpu().numpy())


predictions = np.concatenate(predictions, axis=0)


print(predictions)

[[0.08623526 0.16573855 0.06893624 0.07671241 0.12124654 0.48113102]]


In [28]:
columns = ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']
results_df = pd.DataFrame(predictions, columns=columns)

# 打印 DataFrame
print(results_df)

   seizure_vote  lpd_vote  gpd_vote  lrda_vote  grda_vote  other_vote
0      0.086235  0.165739  0.068936   0.076712   0.121247    0.481131


In [24]:
test_data = pd.read_csv(test_path)
sub = pd.DataFrame({'eeg_id':test_data.eeg_id.values})
sub.head()

,eeg_id
0,3911565283
